In [ ]:
from pyspark.sql import SparkSession
import getpass
username = getpass.getuser()
spark = SparkSession. \
builder. \
config('spark.ui.port','0'). \
config("spark.sql.warehouse.dir", f"/user/itv020649/warehouse"). \
enableHiveSupport(). \
master('yarn'). \
getOrCreate()

In [ ]:
spark

In [3]:
orderDf = spark.read \
.format("csv") \
.option("inferSchema","true") \
.option("header","true") \
.load("/public/trendytech/orders_wh/*")

# shortcut method

In [4]:
spark.read \
.csv("/public/trendytech/orders_wh/*",header ="true", inferSchema = "true")

order_id,order_date,customer_id,order_status
1,2013-07-25 00:00:...,11599,CLOSED
2,2013-07-25 00:00:...,256,PENDING_PAYMENT
3,2013-07-25 00:00:...,12111,COMPLETE
4,2013-07-25 00:00:...,8827,CLOSED
5,2013-07-25 00:00:...,11318,COMPLETE
6,2013-07-25 00:00:...,7130,COMPLETE
7,2013-07-25 00:00:...,4530,COMPLETE
8,2013-07-25 00:00:...,2911,PROCESSING
9,2013-07-25 00:00:...,5657,PENDING_PAYMENT
10,2013-07-25 00:00:...,5648,PENDING_PAYMENT


json not need to give inferschema, but no gurentee it would correctly calculate data type

In [5]:
orderJson = spark.read \
.csv("/public/trendytech/datasets/orders.json")

Parquet metadata is embaded in file, very fast

In [6]:
orderParquet = spark.read \
.csv("/public/trendytech/datasets/ordersparquet")

In [7]:
FilteredDf = orderDf.where("customer_id = 11599")

In [8]:
FilteredDf.show()

+--------+--------------------+-----------+------------+
|order_id|          order_date|customer_id|order_status|
+--------+--------------------+-----------+------------+
|       1|2013-07-25 00:00:...|      11599|      CLOSED|
|   11397|2013-10-03 00:00:...|      11599|    COMPLETE|
|   23908|2013-12-20 00:00:...|      11599|    COMPLETE|
|   53545|2014-06-27 00:00:...|      11599|     PENDING|
|   59911|2013-10-17 00:00:...|      11599|  PROCESSING|
+--------+--------------------+-----------+------------+



In [9]:
orderDf.filter("customer_id = 11599")

order_id,order_date,customer_id,order_status
1,2013-07-25 00:00:...,11599,CLOSED
11397,2013-10-03 00:00:...,11599,COMPLETE
23908,2013-12-20 00:00:...,11599,COMPLETE
53545,2014-06-27 00:00:...,11599,PENDING
59911,2013-10-17 00:00:...,11599,PROCESSING


Data frame to table: intercahngable

In [10]:
orders_df = orderDf.createOrReplaceTempView("orders") 

In [11]:
FilteredDf = spark.sql("select * from orders where order_status = 'CLOSED'")

In [12]:
orderDF = spark.read.table("orders")

In [13]:
orders_df = orderDf.createOrReplaceGlobalTempView("orders") 

# Create table

In [14]:
spark.sql("create database if not exists sukum16_retail")

""


In [15]:
spark.sql("show databases").show()

+--------------------+
|           namespace|
+--------------------+
|0000000000000_msdian|
|0000000000000_nav...|
|0000000009874_retail|
|          00000_2_db|
|00000_aitv019314_...|
|       00000assg5_db|
|     00000assign5_db|
|00000assign7_itv0...|
|0000_0spark_tabbl...|
| 0000_cache_spark111|
|        0000a5a5a5a5|
|   0001_av_ivy_tesco|
|          001_retail|
|        003402_hive1|
|    005198_ivy_tesco|
|    005212_ivy_tesco|
| 005222_ivy_practice|
| 005260_ivy_database|
|005876_week5_assi...|
|       005933_retail|
+--------------------+
only showing top 20 rows



In [16]:
spark.sql("show databases").filter("namespace like 'sukum16%'").show()

+--------------+
|     namespace|
+--------------+
|sukum16_retail|
+--------------+



In [17]:
spark.sql("show tables").show()

+--------+-----------------+-----------+
|database|        tableName|isTemporary|
+--------+-----------------+-----------+
| default|            1htab|      false|
| default|   41group_movies|      false|
| default|    4group_movies|      false|
| default|             4tab|      false|
| default|    6_flags_simon|      false|
| default|                a|      false|
| default|               aa|      false|
| default|              abc|      false|
| default|             acid|      false|
| default|            acid1|      false|
| default|     acid_example|      false|
| default|    acid_example1|      false|
| default|    acid_example2|      false|
| default|           adata1|      false|
| default|        adata_ell|      false|
| default|         adata_vr|      false|
| default|    ad_earthquake|      false|
| default|ad_earthquake_par|      false|
| default|           adelta|      false|
| default|       adeltapart|      false|
+--------+-----------------+-----------+
only showing top

In [18]:
spark.sql("use sukum16_retail")

""


In [24]:
spark.sql("select * from orders limit 10")
# the temp table

order_id,order_date,customer_id,order_status
1,2013-07-25 00:00:...,11599,CLOSED
2,2013-07-25 00:00:...,256,PENDING_PAYMENT
3,2013-07-25 00:00:...,12111,COMPLETE
4,2013-07-25 00:00:...,8827,CLOSED
5,2013-07-25 00:00:...,11318,COMPLETE
6,2013-07-25 00:00:...,7130,COMPLETE
7,2013-07-25 00:00:...,4530,COMPLETE
8,2013-07-25 00:00:...,2911,PROCESSING
9,2013-07-25 00:00:...,5657,PENDING_PAYMENT
10,2013-07-25 00:00:...,5648,PENDING_PAYMENT


In [25]:
spark.sql("describe table orders")

col_name,data_type,comment
order_id,int,null
order_date,string,null
customer_id,int,null
order_status,string,null


In [27]:
spark.sql("drop table sukum16_retail.orders")

""


In [28]:
spark.sql("create table sukum16_retail.orders(order_id integer,order_date string,customer_id integer,order_status string)")

""


In [29]:
spark.sql("show tables")

database,tableName,isTemporary
sukum16_retail,orders,false
,orders,true


In [30]:
spark.sql("insert into sukum16_retail.orders select * from orders")

""


In [31]:
spark.sql("select * from sukum16_retail.orders")

order_id,order_date,customer_id,order_status
1,2013-07-25 00:00:...,11599,CLOSED
2,2013-07-25 00:00:...,256,PENDING_PAYMENT
3,2013-07-25 00:00:...,12111,COMPLETE
4,2013-07-25 00:00:...,8827,CLOSED
5,2013-07-25 00:00:...,11318,COMPLETE
6,2013-07-25 00:00:...,7130,COMPLETE
7,2013-07-25 00:00:...,4530,COMPLETE
8,2013-07-25 00:00:...,2911,PROCESSING
9,2013-07-25 00:00:...,5657,PENDING_PAYMENT
10,2013-07-25 00:00:...,5648,PENDING_PAYMENT


In [32]:
spark.sql("describe table sukum16_retail.orders")

col_name,data_type,comment
order_id,int,null
order_date,string,null
customer_id,int,null
order_status,string,null


In [34]:
spark.sql("describe extended sukum16_retail.orders").show(truncate = False)

+----------------------------+-------------------------------------------------------------------------------+-------+
|col_name                    |data_type                                                                      |comment|
+----------------------------+-------------------------------------------------------------------------------+-------+
|order_id                    |int                                                                            |null   |
|order_date                  |string                                                                         |null   |
|customer_id                 |int                                                                            |null   |
|order_status                |string                                                                         |null   |
|                            |                                                                               |       |
|# Detailed Table Information|                  

data=|Location                    |hdfs://m01.itversity.com:9000/user/itv020649/warehouse/sukum16_retail.db/orders| 
there is directory with. db name >table>files
this is managed table(if dropped=metadata+data=gone)